In [3]:
import random as rd

Variant = 4
rd.seed(Variant)

Numbers_of_problems = [rd.sample(range(4),1)[0]+1, rd.sample(range(4),1)[0]+1]
Numbers_of_problems

[2, 3]

2. С использованием библиотеки ncurses реализовать случайное блуждание курсора по экрану, курсор не должен уходить за границы экрана, при соударении с границей, он должен менять цвет.

In [ ]:
public _start
extrn initscrextrn start_color
extrn init_pairextrn getmaxx
extrn getmaxyextrn raw
extrn noechoextrn keypad
extrn stdscrextrn move
extrn getchextrn addch
extrn refreshextrn endwin
extrn timeoutextrn usleep
extrn printwextrn mvaddch
extrn eraseextrn curs_set
section '.bss' writable
    xmax dq 1
 ymax dq 1 palette dq 1
    delay dq ?    buffer db ?
    f db "/dev/random", 0    rand dq ?
section '.text' executable
_start:
    call initscr    xor rdi, rdi
 mov rdi, [stdscr] call getmaxx
    dec rax mov [xmax], rax
 call getmaxy    dec rax
 mov [ymax], rax
    mov rdi, 0    call curs_set
    call start_color
    ; COLOR_BLUE
    mov rdi, 1    mov rsi, 4
    mov rdx, 4    call init_pair
    ; COLOR_MAGENTA
    mov rdi, 2    mov rsi, 5
    mov rdx, 5    call init_pair
    call refresh
 call noecho    call raw
    xor rax, rax
    mov rax, ' '    or rax, 0x100
    mov [palette], rax
    mov rax, 2    mov rdi, f
    mov rsi, 0    syscall
    mov [rand], rax
    xor r9, r9    xor r10, r10

    .loop:        
        mov rdi, r10        mov rsi, r9
        push r9        push r10
        mov rdx, [palette]        call mvaddch
        call refresh
        mov rdi, 1        call timeout
        call getch        cmp rax, 'q'
        je .end        
        mov rax, 0         mov rdi, [rand]
        mov rsi, buffer        mov rdx, 1
        syscall
        pop r10        pop r9
        xor rax, rax        xor rbx, rbx
        xor rdx, rdx        mov al, [buffer]
        mov rbx, 3        div rbx
        sub rdx, 1        add r9, rdx
        xor rdx, rdx        div rbx
        sub rdx, 1        add r10, rdx
        xor rcx, rcx
        cmp r9, 0        jnl @f
        inc r9        inc rcx
        @@:
        cmp r9, [xmax]        jle @f
        dec r9        inc rcx
        @@:
        cmp r10, 0        jnl @f
        inc r10        inc rcx
        @@:
        cmp r10, [ymax]        jle @f
        dec r10        inc rcx
        @@:
        cmp rcx, 0        je .sleep
        mov rax, [palette]        and rax, 0x100
        cmp rax, 0        jne .mag
        mov rax, [palette]        and rax, 0xff
        or rax, 0x100        jmp @f
        .mag:        mov rax, [palette]
        and rax, 0xff        or rax, 0x200
        @@:        mov [palette], rax

        .sleep:        push r10
        push r9        mov rdi, 100000
        call usleep        call erase
        pop r9        pop r10
                jmp .loop
    .end:
    mov rdi, 1    call curs_set
    call endwin    mov rax, 60
    syscall

3. В программе на ассемблере заполнить массив числами 1, 2, 3,... в количестве $N$ (вводится параметром командной строки), создать два дочерних процесса (clone). Один дочерний процесс должен увеличить на единицу все четные числа, другой - уменьшить на единицу все нечетные. По окончанию работы дочерных процессов родительский процесс должен вывести получившийся массив на экран.

In [ ]:
format elf64

public _start

section '.data' writable
    msg db "Enter the size of array (N): ", 0
    newline db 0xa, 0
    result_msg db "Modified array: ", 0

section '.bss' writable
    n_times resq 1  ; Для хранения размера массива (8 байт)
    array resq 100  ; Для хранения массива (100 элементов, можно увеличить по необходимости)

section '.text' executable

_start:
    ; Печатаем сообщение о вводе размера массива
    mov rsi, msg
    call print_str
    call new_line

    ; Получаем параметр командной строки (размер массива N)
    mov rsi, [rsp+8]    ; Параметры командной строки начинаются с rsi (первый параметр — путь к программе)
    call atoi           ; Преобразуем параметр в число и сохраняем в n_times
    mov [n_times], rax

    ; Заполняем массив числами от 1 до N
    mov rdi, [n_times]  ; Количество элементов в массиве
    mov rbx, 1           ; Начальное значение для заполнения массива

fill_array:
    mov [array + rbx - 1], rbx
    inc rbx
    dec rdi
    jnz fill_array

    ; Создаем первый дочерний процесс для увеличения четных чисел
    mov rax, 56         ; Системный вызов clone
    mov rdi, 0          ; Для простоты не используем флаги
    mov rsi, child1
    syscall

    ; Создаем второй дочерний процесс для уменьшения нечетных чисел
    mov rax, 56         ; Системный вызов clone
    mov rdi, 0          ; Для простоты не используем флаги
    mov rsi, child2
    syscall

    ; Ожидаем завершения дочерних процессов
    mov rax, 60         ; Системный вызов exit
    xor rdi, rdi        ; Завершаем работу родительского процесса
    syscall

child1:
    ; Первый дочерний процесс: увеличивает все четные числа
    mov rbx, 0          ; Индекс в массиве
    mov rcx, [n_times]  ; Количество элементов массива

child1_loop:
    mov rdx, [array + rbx] ; Получаем число из массива
    test rdx, 1         ; Проверяем, четное ли число
    jnz child1_skip     ; Если нечетное, пропускаем
    inc rdx             ; Увеличиваем на 1, если четное
    mov [array + rbx], rdx ; Сохраняем обратно в массив
child1_skip:
    inc rbx
    dec rcx
    jnz child1_loop
    ret

child2:
    ; Второй дочерний процесс: уменьшает все нечетные числа
    mov rbx, 0          ; Индекс в массиве
    mov rcx, [n_times]  ; Количество элементов массива

child2_loop:
    mov rdx, [array + rbx] ; Получаем число из массива
    test rdx, 1         ; Проверяем, нечетное ли число
    jz child2_skip      ; Если четное, пропускаем
    dec rdx             ; Уменьшаем на 1, если нечетное
    mov [array + rbx], rdx ; Сохраняем обратно в массив
child2_skip:
    inc rbx
    dec rcx
    jnz child2_loop
    ret

; Функция для вывода строки
print_str:
    mov rax, 0x1        ; Системный вызов write
    mov rdi, 0x1        ; Стандартный вывод
    mov rdx, [rsi-1]    ; Длина строки
    syscall
    ret

; Функция для вывода новой строки
new_line:
    mov rsi, newline
    call print_str
    ret

; Функция для преобразования строки в число (atoi)
atoi:
    xor rax, rax            ; Обнуляем rax (будет числовое значение)
atoi_loop:
    movzx rbx, byte [rsi]   ; Берем символ
    test rbx, rbx            ; Если ноль, конец строки
    jz atoi_done
    sub rbx, '0'             ; Преобразуем в число
    imul rax, rax, 10        ; Умножаем на 10 (сдвиг влево на 1 разряд)
    add rax, rbx             ; Прибавляем текущую цифру
    inc rsi
    jmp atoi_loop
atoi_done:
    ret
